# Notebook 06: Integration Projects

## Introduction
This final notebook provides templates for capstone projects that integrate multiple concepts from the course.

## Project Ideas
1.  **Atmospheric Chemistry**: Model the Chapman cycle and ozone depletion.
2.  **Enzyme Kinetics**: Simulate Michaelis-Menten kinetics with inhibition.
3.  **Oscillating Reactions**: The Lotka-Volterra or Brusselator models.
4.  **Combustion**: Chain branching in H2 + O2.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets

plt.style.use('seaborn-v0_8-darkgrid')

print("Libraries loaded.")

## Template 1: Atmospheric Chemistry (Chapman Cycle)

Reactions:
1. $O_2 + h\nu \xrightarrow{k_1} 2O$
2. $O + O_2 + M \xrightarrow{k_2} O_3 + M$
3. $O_3 + h\nu \xrightarrow{k_3} O_2 + O$
4. $O + O_3 \xrightarrow{k_4} 2O_2$

In [ ]:
def chapman_cycle(k1, k2, k3, k4, days):
    # Time points (seconds)
    t = np.linspace(0, days * 24 * 3600, 1000)
    
    # Initial concentrations (molecules/cm^3)
    O2_0 = 5e12
    O_0 = 0
    O3_0 = 0
    M = 2e13  # Air density
    
    y0 = [O2_0, O_0, O3_0]
    
    def derivatives(y, t):
        O2, O, O3 = y
        
        # Rates
        r1 = k1 * O2
        r2 = k2 * O * O2 * M
        r3 = k3 * O3
        r4 = k4 * O * O3
        
        dO2dt = -r1 - r2 + r3 + 2*r4
        dOdt = 2*r1 - r2 + r3 - r4
        dO3dt = r2 - r3 - r4
        
        return [dO2dt, dOdt, dO3dt]
    
    # Solve ODE
    sol = odeint(derivatives, y0, t)
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.semilogy(t / 3600, sol[:, 1], label='O')
    plt.semilogy(t / 3600, sol[:, 2], label='O3')
    plt.xlabel('Time (hours)')
    plt.ylabel('Concentration (molecules/cm^3)')
    plt.title('Chapman Cycle Simulation')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example rate constants (approximate)
widgets.interact(chapman_cycle, 
                 k1=widgets.FloatLogSlider(min=-12, max=-8, value=-10, description='k1 (Photolysis O2)'),
                 k2=widgets.FloatLogSlider(min=-34, max=-30, value=-33, description='k2 (O3 formation)'),
                 k3=widgets.FloatLogSlider(min=-4, max=-2, value=-3, description='k3 (Photolysis O3)'),
                 k4=widgets.FloatLogSlider(min=-16, max=-14, value=-15, description='k4 (O3 destruction)'),
                 days=widgets.IntSlider(min=1, max=10, value=2, description='Days'));

### ❓ Concept Check

<details>
<summary><strong>Q: Why does the ozone concentration reach a steady state?</strong></summary>

Because the rate of ozone formation (reaction 2) eventually balances the rate of ozone destruction (reactions 3 and 4). This is a dynamic equilibrium maintained by the constant input of solar energy ($h\nu$).
</details>

## Template 2: Enzyme Kinetics (Michaelis-Menten)

$$ E + S \rightleftharpoons ES \rightarrow E + P $$

In [ ]:
def michaelis_menten(k1, k_1, k2, S0):
    t = np.linspace(0, 100, 500)
    E0 = 1.0
    ES0 = 0
    P0 = 0
    
    y0 = [E0, S0, ES0, P0]
    
    def derivatives(y, t):
        E, S, ES, P = y
        
        r1 = k1 * E * S
        r_1 = k_1 * ES
        r2 = k2 * ES
        
        dEdt = -r1 + r_1 + r2
        dSdt = -r1 + r_1
        dESdt = r1 - r_1 - r2
        dPdt = r2
        
        return [dEdt, dSdt, dESdt, dPdt]
    
    sol = odeint(derivatives, y0, t)
    
    plt.figure(figsize=(10, 6))
    plt.plot(t, sol[:, 1], label='Substrate [S]')
    plt.plot(t, sol[:, 2], label='Complex [ES]')
    plt.plot(t, sol[:, 3], label='Product [P]')
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.title('Enzyme Kinetics')
    plt.legend()
    plt.grid(True)
    plt.show()

widgets.interact(michaelis_menten, 
                 k1=widgets.FloatSlider(min=0.1, max=2.0, value=1.0, description='k1 (Bind)'),
                 k_1=widgets.FloatSlider(min=0.1, max=2.0, value=0.5, description='k-1 (Unbind)'),
                 k2=widgets.FloatSlider(min=0.1, max=2.0, value=0.5, description='k2 (React)'),
                 S0=widgets.FloatSlider(min=1.0, max=10.0, value=5.0, description='[S]0'));

### ❓ Concept Check

<details>
<summary><strong>Q: What is the "Steady State Approximation" in enzyme kinetics?</strong></summary>

It assumes that the concentration of the intermediate complex $[ES]$ remains roughly constant during the main part of the reaction ($d[ES]/dt \approx 0$). You can see this in the plot where the orange curve ($[ES]$) stays flat for a while.
</details>

## Template 3: Oscillating Reactions (Lotka-Volterra)

A simple model for oscillating populations (predator-prey) or autocatalytic reactions.

1. $A + X \xrightarrow{k_1} 2X$ (Autocatalysis)
2. $X + Y \xrightarrow{k_2} 2Y$ (Predation)
3. $Y \xrightarrow{k_3} P$ (Decay)

In [ ]:
def lotka_volterra(k1, k2, k3):
    t = np.linspace(0, 50, 1000)
    X0 = 1.0
    Y0 = 0.5
    
    # Assume [A] is constant (pool chemical)
    A = 1.0
    
    y0 = [X0, Y0]
    
    def derivatives(y, t):
        X, Y = y
        
        dXdt = k1*A*X - k2*X*Y
        dYdt = k2*X*Y - k3*Y
        
        return [dXdt, dYdt]
    
    sol = odeint(derivatives, y0, t)
    
    plt.figure(figsize=(10, 6))
    plt.plot(t, sol[:, 0], label='Species X (Prey)')
    plt.plot(t, sol[:, 1], label='Species Y (Predator)')
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.title('Lotka-Volterra Oscillations')
    plt.legend()
    plt.grid(True)
    plt.show()

widgets.interact(lotka_volterra, 
                 k1=widgets.FloatSlider(min=0.1, max=2.0, value=1.0, description='k1'),
                 k2=widgets.FloatSlider(min=0.1, max=2.0, value=1.0, description='k2'),
                 k3=widgets.FloatSlider(min=0.1, max=2.0, value=1.0, description='k3'));

## Template 4: Chemical Reactor Design

Design a reactor to maximize the yield of intermediate B in a series reaction $A \to B \to C$.
- **CSTR**: Continuous Stirred-Tank Reactor
- **PFR**: Plug Flow Reactor

In [ ]:
class ReactorDesigner:
    """Design a chemical reactor for A -> B -> C"""
    
    def __init__(self):
        self.setup_widgets()
        self.setup_display()
        
    def setup_widgets(self):
        self.tau_slider = widgets.FloatSlider(min=0.1, max=10.0, value=1.0, description='Tau (s)')
        self.k1_slider = widgets.FloatSlider(min=0.1, max=5.0, value=1.0, description='k1 (A->B)')
        self.k2_slider = widgets.FloatSlider(min=0.1, max=5.0, value=0.5, description='k2 (B->C)')
        
    def plot(self, tau, k1, k2):
        # CSTR Equations
        # A = A0 / (1 + k1*tau)
        # B = k1*A*tau / (1 + k2*tau)
        
        # PFR Equations
        # A = A0 * exp(-k1*tau)
        # B = A0 * k1/(k2-k1) * (exp(-k1*tau) - exp(-k2*tau))
        
        A0 = 1.0
        
        # Calculate CSTR
        A_cstr = A0 / (1 + k1*tau)
        B_cstr = k1 * A_cstr * tau / (1 + k2*tau)
        C_cstr = A0 - A_cstr - B_cstr
        
        # Calculate PFR
        if abs(k1 - k2) < 1e-5:
            B_pfr = A0 * k1 * tau * np.exp(-k1*tau)
        else:
            B_pfr = A0 * k1 / (k2 - k1) * (np.exp(-k1*tau) - np.exp(-k2*tau))
        A_pfr = A0 * np.exp(-k1*tau)
        C_pfr = A0 - A_pfr - B_pfr
        
        # Plot concentrations vs Tau (Residence Time)
        tau_range = np.linspace(0, 10, 100)
        
        # PFR Profile over time
        if abs(k1 - k2) < 1e-5:
            B_prof = A0 * k1 * tau_range * np.exp(-k1*tau_range)
        else:
            B_prof = A0 * k1 / (k2 - k1) * (np.exp(-k1*tau_range) - np.exp(-k2*tau_range))
            
        # CSTR Profile
        A_cstr_prof = A0 / (1 + k1*tau_range)
        B_cstr_prof = k1 * A_cstr_prof * tau_range / (1 + k2*tau_range)
        
        plt.figure(figsize=(10, 6))
        plt.plot(tau_range, B_prof, 'b-', label='PFR Yield (B)')
        plt.plot(tau_range, B_cstr_prof, 'r--', label='CSTR Yield (B)')
        
        plt.axvline(tau, color='k', linestyle=':', label=f'Current Tau = {tau} s')
        plt.plot(tau, B_pfr, 'bo', markersize=10)
        plt.plot(tau, B_cstr, 'ro', markersize=10)
        
        plt.xlabel('Residence Time (s)')
        plt.ylabel('Concentration of B')
        plt.title('Reactor Design: Maximizing Intermediate B')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        print(f"PFR Yield: {B_pfr:.3f} ({(B_pfr/A0)*100:.1f}%)")
        print(f"CSTR Yield: {B_cstr:.3f} ({(B_cstr/A0)*100:.1f}%)")
        if B_pfr > B_cstr:
            print("PFR is more efficient for this reaction.")
        else:
            print("CSTR is more efficient (unlikely for series reaction).")

    def setup_display(self):
        widgets.interact(self.plot, tau=self.tau_slider, k1=self.k1_slider, k2=self.k2_slider)

print("\n🏭 Chemical Reactor Designer:")
reactor = ReactorDesigner()

## Template 5: Photosynthetic Electron Transfer

Model the Z-scheme of photosynthesis, tracking electron flow from Water to NADP+.

In [ ]:
class PhotosynthesisModel:
    """Simulate Z-Scheme Electron Transfer"""
    
    def __init__(self):
        # Simplified Z-scheme energies (eV vs SHE approx)
        self.components = {
            'P680': 1.2, 'P680*': -0.8,
            'QA': -0.1, 'QB': 0.0,
            'PQ': 0.1, 'Cytb6f': 0.4,
            'PC': 0.5,
            'P700': 0.6, 'P700*': -1.2,
            'FA/FB': -0.5, 'Fd': -0.4, 'NADP+': -0.32
        }
        self.setup_display()
        
    def plot_z_scheme(self, light_intensity):
        # Adjust excited state populations based on light
        pop_P680_star = min(1.0, light_intensity / 100)
        pop_P700_star = min(1.0, light_intensity / 100)
        
        # Plot Energy Diagram
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Coordinates for components
        x_coords = np.arange(len(self.components))
        energies = list(self.components.values())
        names = list(self.components.keys())
        
        # Draw levels
        for i, (name, E) in enumerate(self.components.items()):
            color = 'orange' if '*' in name else 'green'
            ax.plot([i-0.3, i+0.3], [E, E], color=color, linewidth=3)
            ax.text(i, E+0.05, name, ha='center')
            
        # Draw connections (Electron Flow)
        # P680 -> P680* (Light)
        ax.annotate('', xy=(1, self.components['P680*']), xytext=(0, self.components['P680']),
                   arrowprops=dict(arrowstyle='->', color='yellow', lw=2, linestyle='--'))
        
        # P680* -> QA -> ... -> P700
        path_indices = [1, 2, 3, 4, 5, 6, 7]
        for i in range(len(path_indices)-1):
            idx1, idx2 = path_indices[i], path_indices[i+1]
            ax.annotate('', xy=(idx2, energies[idx2]), xytext=(idx1, energies[idx1]),
                       arrowprops=dict(arrowstyle='->', color='blue'))
            
        # P700 -> P700* (Light)
        ax.annotate('', xy=(8, self.components['P700*']), xytext=(7, self.components['P700']),
                   arrowprops=dict(arrowstyle='->', color='yellow', lw=2, linestyle='--'))
                   
        # P700* -> ... -> NADP+
        path_indices_2 = [8, 9, 10, 11]
        for i in range(len(path_indices_2)-1):
            idx1, idx2 = path_indices_2[i], path_indices_2[i+1]
            ax.annotate('', xy=(idx2, energies[idx2]), xytext=(idx1, energies[idx1]),
                       arrowprops=dict(arrowstyle='->', color='blue'))
        
        ax.set_ylabel('Redox Potential (V)')
        ax.set_title(f'Z-Scheme Photosynthesis (Light Intensity: {light_intensity}%)')
        ax.invert_yaxis() # Convention in biology: negative (high energy) up
        ax.grid(True, alpha=0.3)
        plt.show()
        
        # Calculate theoretical efficiency
        input_energy = 2 * 1.8 # approx eV for 680nm and 700nm photons
        stored_energy = 1.2 - (-0.32) # Delta E from H2O to NADP+
        eff = (stored_energy / input_energy) * 100
        print(f"Theoretical Energy Efficiency: {eff:.1f}%")
        print(f"Rate of NADPH production is proportional to light intensity: {light_intensity}%")

    def setup_display(self):
        widgets.interact(self.plot_z_scheme, 
                        light_intensity=widgets.IntSlider(min=0, max=100, step=10, value=50, description='Light %'))

print("\n🌿 Photosynthesis Z-Scheme Explorer:")
photo_model = PhotosynthesisModel()

## Template 6: Build Your Own Reaction (PES Constructor)

Create a custom Potential Energy Surface (PES) by adding Gaussian wells and hills, then simulate a reaction trajectory.

In [ ]:
class PESConstructor:
    """Interactive 2D Potential Energy Surface Builder"""
    
    def __init__(self):
        self.wells = [] # List of (x, y, depth, width)
        self.setup_widgets()
        
    def add_well(self, x, y, depth, width):
        self.wells.append({'x': x, 'y': y, 'depth': depth, 'width': width})
        
    def clear(self):
        self.wells = []
        
    def potential(self, X, Y):
        Z = np.zeros_like(X)
        # Default harmonic well
        Z += 0.5 * (X**2 + Y**2)
        
        # Add custom Gaussian wells/hills
        for w in self.wells:
            # Gaussian: A * exp(-((x-x0)^2 + (y-y0)^2) / 2w^2)
            Z += w['depth'] * np.exp(-((X - w['x'])**2 + (Y - w['y'])**2) / (2 * w['width']**2))
            
        return Z
        
    def plot(self, n_wells):
        # Just for interactivity, we regenerate random wells if n_wells changes
        # In a real app, we'd have click-to-add
        if len(self.wells) != n_wells:
            self.clear()
            for _ in range(n_wells):
                self.add_well(np.random.uniform(-2, 2), np.random.uniform(-2, 2), 
                             np.random.uniform(-5, 5), np.random.uniform(0.5, 1.0))
        
        x = np.linspace(-3, 3, 100)
        y = np.linspace(-3, 3, 100)
        X, Y = np.meshgrid(x, y)
        Z = self.potential(X, Y)
        
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        surf = ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
        fig.colorbar(surf, shrink=0.5, aspect=5)
        
        ax.set_title(f'Custom Potential Energy Surface ({n_wells} Features)')
        ax.set_xlabel('Coordinate 1')
        ax.set_ylabel('Coordinate 2')
        ax.set_zlabel('Energy')
        plt.show()
        
        # Run a simple "ball" trajectory from (2, 2)
        # Gradient descent
        path_x, path_y = [2.0], [2.0]
        curr_x, curr_y = 2.0, 2.0
        lr = 0.1
        for _ in range(20):
            # Finite difference gradient
            eps = 0.01
            dzdx = (self.potential(curr_x+eps, curr_y) - self.potential(curr_x-eps, curr_y)) / (2*eps)
            dzdy = (self.potential(curr_x, curr_y+eps) - self.potential(curr_x, curr_y-eps)) / (2*eps)
            
            curr_x -= lr * dzdx
            curr_y -= lr * dzdy
            path_x.append(curr_x)
            path_y.append(curr_y)
            
        print("Trajectory simulation (Gradient Descent) from (2,2) completed.")
        print(f"Final position: ({curr_x:.2f}, {curr_y:.2f})")

    def setup_display(self):
        widgets.interact(self.plot, n_wells=widgets.IntSlider(min=0, max=5, value=2, description='Features'))

print("\n🏔️ PES Constructor (Randomized Features):")
pes_builder = PESConstructor()